In [1]:
    import pickle as plk
with open("liwc_categories_for_vocabulary_erisk_clpsych_stop_20K.pkl", "rb") as f:
    data = plk.load(f)

In [2]:
data["sexual"]

{'abortion',
 'abortions',
 'aids',
 'ass',
 'asses',
 'bi',
 'boner',
 'boob',
 'boobies',
 'boobs',
 'breast',
 'breastfeeding',
 'breasts',
 'butt',
 'butts',
 'cock',
 'cocks',
 'condom',
 'condoms',
 'cuddle',
 'cuddled',
 'cuddles',
 'cuddling',
 'cuddly',
 'dick',
 'dicks',
 'dyke',
 'erection',
 'erotic',
 'fuck',
 'fucked',
 'fucker',
 'fuckers',
 'fuckery',
 'fuckin',
 'fucking',
 'fucks',
 'gay',
 'gays',
 'genital',
 'genitals',
 'hiv',
 'ho',
 'homo',
 'homosexual',
 'homosexuality',
 'homosexuals',
 'horny',
 'hug',
 'hugged',
 'hugging',
 'hugs',
 'hump',
 'incest',
 'kiss',
 'kissed',
 'kisses',
 'kissing',
 'lesbian',
 'lesbians',
 'love',
 'loved',
 'lover',
 'lovers',
 'loves',
 'lust',
 'naked',
 'nipple',
 'nipples',
 'nude',
 'nudes',
 'orgasm',
 'orgasms',
 'orgy',
 'ovaries',
 'passion',
 'passionate',
 'passionately',
 'passions',
 'penis',
 'penises',
 'pervert',
 'porn',
 'pornhub',
 'porno',
 'pornography',
 'pregnancy',
 'pregnant',
 'prostate',
 'prostitut

In [41]:
num2emo = {}
whole_words = {}
asterisk_words = {}

with open("../liwc.dic", "r") as f:
    opening_tag = False
    for line in f:
        line = line.strip()
        if line == "%":
            opening_tag = not opening_tag
            continue
        if opening_tag:
            _id, _name = line.split("\t")
            num2emo[_id] = [_name]
        elif "*" not in line:
            _name, *_categories = line.split("\t")
            whole_words[_name] = _categories
        elif "*" in line and not line.startswith("*"):
            _name, *_categories = line.split("\t")
            _chars = list(_name)
            pointer = asterisk_words
            for idx, char in enumerate(_chars):
                if char == "*":
                    pointer["*"] = _categories
                else:
                    if char not in pointer:
                        pointer[char] = {}
                    pointer = pointer[char]
        else:
            raise Exception(f"{line}")

In [42]:
class LIWC_vectorizer(object):
    def __init__(self, num2emo, whole_words, asterisk_words):
        self.num2emo = num2emo
        self.num2idx = {num: idx for idx, num in enumerate(self.num2emo.keys())}
        self.whole_words = whole_words
        self.asterisk_words = asterisk_words

    def get_vector(self, tokens, relative=True):
        categories_cnt = [0] * len(self.num2idx)
        for word in tokens:
            categories = whole_words.get(word, None)
            if categories is None:
                chars = list(word)
                pointer = self.asterisk_words
                for ch in chars:
                    if ch in pointer:
                        pointer = pointer[ch]
                if "*" in pointer:
                    categories = pointer["*"]
                else:
                    # not found
                    categories = []
            for idx_category in categories:
                categories_cnt[self.num2idx[idx_category]] += 1
        text_len = len(tokens)
        if relative:
            for i in range(len(self.num2idx)):
                categories_cnt[i] = categories_cnt[i] / text_len
        return categories_cnt

vec = LIWC_vectorizer(num2emo, whole_words, asterisk_words)
vec.get_vector(["advis", "actually", "xxxx"])

[0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]